Import Libraries

In [1]:
import pandas as pd
import seaborn as sns
import heapq
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score

Download Dataset

In [2]:
!wget --no-check-certificate \
    "https://drive.google.com/uc?export=download&id=11AFzQrlWoGF9SqFrDGGS65oP9L48Ajg4" \
    -O "/content/diabetes_dataset__2019.csv"

--2023-12-03 07:01:51--  https://drive.google.com/uc?export=download&id=11AFzQrlWoGF9SqFrDGGS65oP9L48Ajg4
Resolving drive.google.com (drive.google.com)... 142.251.2.139, 142.251.2.101, 142.251.2.113, ...
Connecting to drive.google.com (drive.google.com)|142.251.2.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-04-b4-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/qvtbbeh6po1deda6irp1frmogb37gspn/1701586875000/03346498828059726724/*/11AFzQrlWoGF9SqFrDGGS65oP9L48Ajg4?e=download&uuid=0b78248f-99ed-4c35-88ef-cc54a4f70649 [following]
--2023-12-03 07:01:52--  https://doc-04-b4-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/qvtbbeh6po1deda6irp1frmogb37gspn/1701586875000/03346498828059726724/*/11AFzQrlWoGF9SqFrDGGS65oP9L48Ajg4?e=download&uuid=0b78248f-99ed-4c35-88ef-cc54a4f70649
Resolving doc-04-b4-docs.googleusercontent.com (doc-04-b4-docs.googleusercontent.com)... 142.251.2.132, 2607:f8b0:

Explore Dataset

In [3]:
diabetes_df = pd.read_csv("/content/diabetes_dataset__2019.csv")

diabetes_df.head()
diabetes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 952 entries, 0 to 951
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Age               952 non-null    object 
 1   Gender            952 non-null    object 
 2   Family_Diabetes   952 non-null    object 
 3   highBP            952 non-null    object 
 4   PhysicallyActive  952 non-null    object 
 5   BMI               948 non-null    float64
 6   Smoking           952 non-null    object 
 7   Alcohol           952 non-null    object 
 8   Sleep             952 non-null    int64  
 9   SoundSleep        952 non-null    int64  
 10  RegularMedicine   952 non-null    object 
 11  JunkFood          952 non-null    object 
 12  Stress            952 non-null    object 
 13  BPLevel           952 non-null    object 
 14  Pregancies        910 non-null    float64
 15  Pdiabetes         951 non-null    object 
 16  UriationFreq      952 non-null    object 
 1

Data Cleaning

In [4]:
# see the value of each column
print('Value of each column')
for col in diabetes_df.columns:
    print(col)
    print(diabetes_df[col].value_counts())
    print("\n")

# remove unused data
diabetes_df.drop('SoundSleep',axis=1,inplace=True)
diabetes_df.drop('highBP',axis=1,inplace=True)
diabetes_df.drop('UriationFreq',axis=1,inplace=True)
diabetes_df.drop('Stress',axis=1,inplace=True)
diabetes_df.drop('JunkFood',axis=1,inplace=True)
diabetes_df.drop('BPLevel',axis=1,inplace=True)

# remove empty data
diabetes_df = diabetes_df[diabetes_df['BMI'].notna()]
diabetes_df = diabetes_df[diabetes_df['Pregancies'].notna()]
diabetes_df = diabetes_df[diabetes_df['Diabetic'].notna()]
diabetes_df = diabetes_df[diabetes_df['Pdiabetes'].notna()]

# Age
diabetes_df['Age'].replace(['less than 40', '40-49', '50-59','60 or older'], ['0','1','2','3',], inplace=True)

# Gender
diabetes_df['Gender'].replace(['Male','Female'],['0','1'], inplace=True)

#Family_Diabetes
diabetes_df['Family_Diabetes'].replace(['no', 'yes'],['0','1'], inplace=True)

# PhysicallyActive
diabetes_df['PhysicallyActive'].replace(['none', 'less than half an hr', 'more than half an hr', 'one hr or more'], ['0','1','2','3'], inplace=True)

# Smoking
diabetes_df['Smoking'].replace(['no','yes'],['0','1'], inplace=True)

# Alcohol
diabetes_df['Alcohol'].replace(['no','yes'],['0','1'], inplace=True)

# RegularMedicine
diabetes_df['RegularMedicine'].replace(['no','o','yes'],['0','0','1'], inplace=True)

# Pdiabetes
diabetes_df['Pdiabetes'].replace(['0','yes'],['0','1'], inplace=True)

# Diabetic
diabetes_df['Diabetic'].replace(['no',' no','yes'],['0','0','1'], inplace=True)

Value of each column
Age
less than 40    488
40-49           164
50-59           156
60 or older     144
Name: Age, dtype: int64


Gender
Male      580
Female    372
Name: Gender, dtype: int64


Family_Diabetes
no     498
yes    454
Name: Family_Diabetes, dtype: int64


highBP
no     724
yes    228
Name: highBP, dtype: int64


PhysicallyActive
less than half an hr    336
more than half an hr    272
one hr or more          212
none                    132
Name: PhysicallyActive, dtype: int64


BMI
24.0    111
21.0     88
23.0     76
28.0     71
26.0     66
33.0     64
27.0     63
22.0     58
20.0     48
19.0     36
25.0     34
30.0     33
18.0     32
29.0     28
38.0     28
36.0     20
17.0     16
32.0     16
31.0     16
34.0     12
35.0     12
15.0      8
39.0      4
40.0      4
42.0      3
45.0      1
Name: BMI, dtype: int64


Smoking
no     844
yes    108
Name: Smoking, dtype: int64


Alcohol
no     760
yes    192
Name: Alcohol, dtype: int64


Sleep
7     264
8     256
6     256
5    

Split Dataframe: Diabetic and Non-diabetic

In [5]:
diabetic_mask = diabetes_df['Diabetic'] == '1'
diabetic_df = diabetes_df[diabetic_mask]

not_diabetic_mask = diabetes_df['Diabetic'] == '0'
not_diabetic_df = diabetes_df[not_diabetic_mask]

diabetic_df.to_csv('diab_clean.csv', encoding='utf-8', index=False)
diabetic_clean_df = pd.read_csv('diab_clean.csv')
diabetic_clean_df['Diabetic'] = diabetic_clean_df['Diabetic'].fillna(1)

not_diabetic_df.to_csv('not_diab_clean.csv', encoding='utf-8', index=False)
not_diabetic_clean_df = pd.read_csv('not_diab_clean.csv')
not_diabetic_clean_df['Diabetic'] = not_diabetic_clean_df['Diabetic'].fillna(0)

frames = [diabetic_clean_df, not_diabetic_clean_df]
diabetes_clean_df = pd.concat(frames)
diabetes_clean_df.to_csv('diabetes_clean_df.csv', encoding='utf-8', index=False)

diab_df = pd.read_csv('diab_clean.csv')
not_diab_df = pd.read_csv('not_diab_clean.csv')

Model Tensorflow

In [6]:
# Load the cleaned dataset
diabetes_clean_df = pd.read_csv('diabetes_clean_df.csv')

# Extract features and target variable
choices = ["Gender", "Family_Diabetes", "PhysicallyActive", "BMI", "Pdiabetes", "Smoking", "Alcohol", "Sleep", "RegularMedicine", "Pregancies"]
X = diabetes_clean_df[choices]
y = diabetes_clean_df['Diabetic']

# Convert categorical variables to one-hot encoding
X = pd.get_dummies(X, columns=["Gender", "Family_Diabetes", "PhysicallyActive", "Smoking", "Alcohol", "RegularMedicine"])

# Standardize numerical features
scaler = StandardScaler()
X[['BMI', 'Sleep', 'Pregancies']] = scaler.fit_transform(X[['BMI', 'Sleep', 'Pregancies']])

# Split the dataset into training and testing sets
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=2)

# Build the TensorFlow model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

# Evaluate the model on the test set
y_pred_proba = model.predict(X_test)
y_pred = (y_pred_proba > 0.5).astype(int)

# Calculate metrics
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)

# Print the metrics
print("Accuracy: ", acc)
print("Precision: ", prec)
print("Recall: ", rec)

Epoch 1/10
21/21 [==============================] - 2s 23ms/step - loss: 0.6485 - accuracy: 0.6605 - val_loss: 0.5504 - val_accuracy: 0.8356
Epoch 2/10
21/21 [==============================] - 0s 9ms/step - loss: 0.5473 - accuracy: 0.7419 - val_loss: 0.4419 - val_accuracy: 0.8356
Epoch 3/10
21/21 [==============================] - 0s 7ms/step - loss: 0.4685 - accuracy: 0.7896 - val_loss: 0.3735 - val_accuracy: 0.8630
Epoch 4/10
21/21 [==============================] - 0s 8ms/step - loss: 0.4142 - accuracy: 0.8233 - val_loss: 0.3375 - val_accuracy: 0.8493
Epoch 5/10
21/21 [==============================] - 0s 8ms/step - loss: 0.3841 - accuracy: 0.8372 - val_loss: 0.3267 - val_accuracy: 0.8904
Epoch 6/10
21/21 [==============================] - 0s 5ms/step - loss: 0.3648 - accuracy: 0.8356 - val_loss: 0.3207 - val_accuracy: 0.9041
Epoch 7/10
21/21 [==============================] - 0s 10ms/step - loss: 0.3510 - accuracy: 0.8433 - val_loss: 0.3149 - val_accuracy: 0.9041
Epoch 8/10
21/21 [

In [11]:
# Save the original TensorFlow model
model.save("model.h5")
# Convert the TensorFlow model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
# Save the TensorFlow Lite model to a file
with open("model.tflite", "wb") as f:
    f.write(tflite_model)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
